# Introduction: Book Recommendation System

In this notebook, we will build a book recommendation system based on a simple principle: books that link to the same Wikipedia pages are alike. In order to create this representation of similar books, we'll use the concept of neural network entity embeddings, mapping each book and each Wikipedia link (Wikilink) to a 50 number vector. The idea of entity embeddings is to map high-dimensional categorical variables to a low-dimensional _learned_ vector that _places similar entities closer together in the embedding space_. If we were to one-hot-encode the books (another representation of categorical data) we would have a 37,000 dimension vector for each book, with a single 1 indicating the book and similar books would not be "closer" to one another. By  training a neural network to learn entity embeddings, we not only get a reduced dimension representation of the books, we also get a representation that _keeps similar books closer to each other_. Therefore, the basic approach is to create entity embeddings of all the books, and then for any book, find the closest book in the embedding space as a recommendation. [Fortunately, we have access to every single book on Wikipedia](https://github.com/WillKoehrsen/wikipedia-data-science/blob/master/Downloading%20and%20Parsing%20Wikipedia%20Articles.ipynb), which will let us create a comprehensive system.

## Approach

To create an entity embedding, we train a neural network for a supervised machine learning task and tell it to represent the books and the links as vectors that can be adjusted in order to improve the predictions. In other words, the parameters of the model are the embeddings meaning that the network tries to learn the task by changing the representations of the books and the links. Once we have the embeddings for the books and the links, we can find the most similar book to a given book by computing the distance between the vector for that book and all the other books. We'll use the cosine distance which measures the angle between two vectors as a measure of similarity (another valid option is the Euclidean distance). We can also do the same with the links, finding the most similar page to a given page that occurs in the links.

### Supervised Machine Learning Task: Map Books to Links

For our machine learning task, we'll set up the problem as identifying whether or not a particular link was present in a book article. The training examples will consist of (book, link) pairs, with some pairs true examples - actually  in the data - and others negative examples that do not occur in the data. It will be the network's job to adjust the entity embeddings of the books and the links in order to accurately make this classification.

## Entity Embeddings

Entity embeddings have proven to be very powerful concepts for modeling language and categorical variables. For example, the Word2Vec embeddings map a word to a 50 dimensional vector based on training a neural network on millions of words. Embeddings can then be used in any supervised model, because they are just representations of categorical variables. Much as we might one-hot-encode categorical variables to use them in a random forest for a supervised task, we can also use the entity embeddings to include categorical variables in any model. 

We can also use the Entity Embeddings to visualize concepts, such as creating a map of all books on Wikipedia. The entity embeddings typically are still high-dimensional - we'll use 50 numbers for each entity - so we need to use a dimension reduction technique such as TSNE or UMAP to visualize the embeddings in lower dimensions. We'll take a look at doing this at the end of the notebook and later will upload the embeddings into a application custom-built for this purpose ([projector.tensorflow.org](https://projector.tensorflow.org)). Entity embeddings are becoming more widespread thanks to the ease of development of neural networks in Keras and are a worthwhile approach when we want to represent categorical variables in relevant numbers. Other approaches for encoding categorical variables do not represent similar entities as being closer to one another, and entity embedding is a learning-based method for this crucial task.

Overall, this project is a great look at the potential for neural networks to create meaningful embeddings of high dimensional data and a practical application of deep learning. The code itself is relatively simple, and the Keras library makes developing deep learning models enjoyable!

The code here is adapted from the excellent [Deep Learning Cookbook](http://shop.oreilly.com/product/0636920097471.do), the [notebooks for which can be found on GitHub](https://github.com/DOsinga/deep_learning_cookbook). Check out this book for practical applications of deep learning and great projects! 

## Read in Data

The data is stored as json with line for every book. This data contains every single book article on Wikipedia which was parsed in the [Downloading and Parsing Wikipedia Data Notebook](https://github.com/WillKoehrsen/wikipedia-data-science/blob/master/Downloading%20and%20Parsing%20Wikipedia%20Articles.ipynb).

In [1]:
from IPython.core.interactiveshell import InteractiveShell

# Set shell to show all lines of output
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import json

books = []

with open('found_books_filtered.ndjson', 'r') as fin:
    # Append each line to the books
    books = [json.loads(l) for l in fin]

# Remove non-book articles
books_with_wikipedia = [book for book in books if 'Wikipedia:' in book[0]]
books = [book for book in books if 'Wikipedia:' not in book[0]]
print(f'Found {len(books)} books.')

Found 37020 books.


There are a few articles that were caught which are clearly not books (feel free to check out these articles yourself).

In [3]:
[book[0] for book in books_with_wikipedia][:5]

['Wikipedia:Wikipedia Signpost/2014-06-25/Recent research',
 'Wikipedia:New pages patrol/Unpatrolled articles/December 2010',
 'Wikipedia:Templates for discussion/Log/2012 September 23',
 'Wikipedia:Articles for creation/Redirects/2012-10',
 'Wikipedia:Templates for discussion/Log/2012 October 4']

Each legitimate book contains the __title, the information from the `Infobox book` template, the internal wikipedia links, the external links, the date of last edit, and the number of characters in the article__ (a rough estimate of the length of the article).

In [37]:
n = 21
books[n][0], books[n][1], books[n][2][:5], books[n][3][:5], books[n][3][:5], books[n][4], books[n][5]

('Limonov (novel)',
 {'author': 'Emmanuel Carrère',
  'country': 'France',
  'english_pub_date': '2014',
  'isbn': '978-2-8180-1405-9',
  'language': 'French',
  'name': 'Limonov',
  'pages': '488',
  'pub_date': '2011',
  'publisher': 'P.O.L.',
  'translator': 'John Lambert'},
 ['Emmanuel Carrère',
  'biographical novel',
  'Emmanuel Carrère',
  'Eduard Limonov',
  'Prix de la langue française'],
 ['http://www.lefigaro.fr/flash-actu/2011/10/05/97001-20111005FILWWW00615-le-prix-de-la-langue-francaise-a-e-carrere.php',
  'http://www.lexpress.fr/culture/livre/emmanuel-carrere-prix-renaudot-2011_1046819.html',
  'http://limonow.de/carrere/index.html',
  'http://www.tout-sur-limonov.fr/222318809'],
 ['http://www.lefigaro.fr/flash-actu/2011/10/05/97001-20111005FILWWW00615-le-prix-de-la-langue-francaise-a-e-carrere.php',
  'http://www.lexpress.fr/culture/livre/emmanuel-carrere-prix-renaudot-2011_1046819.html',
  'http://limonow.de/carrere/index.html',
  'http://www.tout-sur-limonov.fr/222318

## Map Books to Integers

First we want to create a mapping of books to integers. When we feed books into the neural network for the embedding, we will have to represent them as numbers, so this mapping will let us keep track of the books. We'll also create the reverse mapping, from integers back to the title.

In [39]:
book_index = {book[0]: idx for idx, book in enumerate(books)}
index_book = {idx: book for book, idx in book_index.items()}

book_index['Anna Karenina']
index_book[22494]

22494

'Anna Karenina'

### Exploring Wikilinks

Although it's not our main focus, we can do a little exploration. Let's find the number of unique Wikilinks  and the most common ones. To create a single list from a list of lists, we can use the `itertools` chain method.

In [40]:
from itertools import chain

wikilinks = list(chain(*[book[2] for book in books]))
print(f"There are {len(set(wikilinks))} unique wikilinks.")

There are 311276 unique wikilinks.


How many of these are links to other books? 

In [41]:
wikilinks_other_books = [link for link in wikilinks if link in book_index.keys()]
print(f"There are {len(set(wikilinks_other_books))} unique links to other books.")

There are 17032 unique links to other books.


Let's take a look at the counts of each wikilink. We'll make a utility function that takes in a list and returns a sorted ordered dictionary of the counts of the items in the list. The `collections` module has a number of useful functions for dealing with groups of objects.

In [42]:
from collections import Counter, OrderedDict

def count_items(l):
    """Return ordered dictionary of counts of objects in `l`"""
    
    # Create a counter object
    counts = Counter(l)
    
    # Sort by highest count first and place in ordered dictionary
    counts = sorted(counts.items(), key = lambda x: x[1], reverse = True)
    counts = OrderedDict(counts)
    
    return counts

In [43]:
wikilink_counts = count_items(wikilinks)
list(wikilink_counts.items())[:10]

[('Hardcover', 7641),
 ('Paperback', 7482),
 ('Wikipedia:WikiProject Books', 6116),
 ('Wikipedia:WikiProject Novels', 6087),
 ('English language', 4307),
 ('The New York Times', 3960),
 ('United States', 3388),
 ('Science fiction', 3121),
 ('science fiction', 2661),
 ('Publishers Weekly', 2414)]

The most linked to pages are in fact not that surprising! One thing we should notice is that there are discrepancies in capitalization. We want to normalize across capitalization, so we'll lowercase all of the links and redo the counts.


### Data Cleaning of Wikilinks

In [44]:
wikilinks = [link.lower() for link in wikilinks]
print(f"There are {len(set(wikilinks))} unique wikilinks.")

wikilink_counts = count_items(wikilinks)
list(wikilink_counts.items())[:10]

There are 297624 unique wikilinks.


[('paperback', 8976),
 ('hardcover', 8854),
 ('wikipedia:wikiproject books', 6116),
 ('wikipedia:wikiproject novels', 6088),
 ('science fiction', 5917),
 ('english language', 4371),
 ('the new york times', 3976),
 ('united states', 3391),
 ('novel', 3044),
 ('publishers weekly', 2414)]

That actually changes the rankings! This illustrates an important point: __make sure to take a look at your data before modeling!__ 

I'm going to remove the __most popular__ wikilinks because these are not very informative. Knowing whether a book is hardcover or paperback is not that important to the content. We also don't need the two `Wikipedia:` links since these do not distinguish the books based on content. __I'd recommend playing around with this list because it might have a large effect on the recommendations.__

In [46]:
to_remove = ['hardcover', 'paperback', 'wikipedia:wikiproject books', 'wikipedia:wikiproject novels']
for t in to_remove:
    links.remove(t)
    _ = wikilink_counts.pop(t)

Since there are so many unique wikilinks, I'm going to limit the list to wikilinks mentioned 5 or more times. Hopefully this reduces the noise that might come from wikilinks that only appear a few times. Keeping every single link leads to unreasonable neural network training time, but you are welcome to try!

In [47]:
# Limit to greater than 4 links
links = [t[0] for t in wikilink_counts.items() if t[1] >= 5]
print(len(links))

38550


As a final bit of exploration, let's look at the books that are mentioned the most by other books on Wikipedia. 

In [58]:
wikilink_books_counts = count_items(wikilinks_other_books)
list(wikilink_books_counts.items())[:10]

[('The Discontinuity Guide', 148),
 ('The Encyclopedia of Science Fiction', 145),
 ('Dracula', 72),
 ('The Dresden Files', 70),
 ('The Encyclopedia of Fantasy', 66),
 ('Encyclopædia Britannica', 61),
 ('The Wonderful Wizard of Oz', 61),
 ('Nineteen Eighty-Four', 59),
 ("Alice's Adventures in Wonderland", 54),
 ('Don Quixote', 52)]

It's not surprising that several of these are references. We also see that a few classics make it into the list! 

## Wikilinks to Index

As with the books, we need to map the Wikilinks to integers. We'll also create the reverse mapping.

In [60]:
link_index = {link: idx for idx, link in enumerate(links)}
index_link = {idx: link for link, idx in link_index.items()}

link_index['the economist']
index_link[240]
len(link_index)

240

'the economist'

38550

# Build a Training Set

In order for any machine learning model to learn, it needs a training set. We are going to treat this as a supervised learning problem: given a pair (book, link), we want the neural network to learn to predict whether this is a legitimate pair - present in the data - or not.

For each book, we'll iterate through the wikilinks on the book page and save them to a list. The final `pairs` list will consist of tuples of every (book, link) pairing on all of Wikipedia.

In [17]:
pairs = []

# Iterate through each book
for book in books:
    # Iterate through the links in the book
    pairs.extend((book_index[book[0]], link_index[link.lower()]) for link in book[2] if link.lower() in links)
    
len(pairs), len(links), len(books)

(768821, 38550, 37020)

We now have over 750,000 positive examples on which to train! Each pair represents one Wikilink for one book. Let's look at a few examples.

In [61]:
index_book[pairs[5000][0]], index_link[pairs[5000][1]]

("Dr. Seuss's ABC", 'dr. seuss')

In [63]:
index_book[pairs[900][0]], index_link[pairs[900][1]]

('The Man Who Watched the Trains Go By (novel)',
 'category:belgian novels adapted into films')

Later on we'll create the negative examples by randomly sampling from the links and the books and making sure the resulting pair is not in `pairs`. 

In [64]:
pairs_set = set(pairs)

## Note about Training / Testing Set

To compute the embeddings, we are not going to create a separate validation or testing set. While this is a __must__ for a normal supervised machine learning task, in this case, our primary objective is not to make the most accurate model, but to generate the best embeddings. The prediction task is just the method through which we train our network to make the embeddings. At the end of training, we are not going to be testing our model on new data, so we don't need to evaluate the performance. Instead of testing on new data, we'll look at the embeddings themselves to see if books that we think are similar have embeddings that are close to each other. 

If we kept a separate validation / testing set, then we would be limiting the amount of data that our network can use to train. This would result in less accurate embeddings. Normally with any supervised model, we need to be concerned about overfitting, but again, because we do not need our model to generalize to new data and our goal is the embeddings, we will make our model as effective as possible by using all the data for training. In general, always have a separate validation and testing set (or use cross validation) and make sure to regularize your model to prevent overfitting. 

Just for fun, let's look at the (book, link) pairs that are represented most often in the data. 

In [65]:
x = Counter(pairs)
sorted(x.items(), key = lambda x: x[1], reverse = True)[:5]

[((13337, 725), 85),
 ((31899, 40), 77),
 ((5919, 1884), 62),
 ((25899, 1395), 61),
 ((10394, 1955), 60)]

In [68]:
index_book[13337], index_link[725]
index_book[31899], index_link[40]
index_book[10394], index_link[1955]

("France's Songs of the Bards of the Tyne - 1850", 'joseph philip robson')

('The Early Stories: 1953–1975', 'the new yorker')

('Registry of World Record Size Shells', 'gastropoda')

There's nothing wrong with books that link to the same page many times. They are just more likely to be trained on since there are more of them.

## Generator For Training Samples

We want to generate positive samples and negative samples. The positive samples are simple: pick one random pair from the `pairs` and assign it a 1. The negative samples are also fairly easy: pick one random link and one random book, make sure they are not in the `pairs`, and assign them a -1 or a 0. We'll use either a 0 or -1 for the negative labels depending on whether we want to make this a regression or a classification problem. Either approach is valid, and we'll try out both methods.

The code below creates a generator that yields batches of samples. Each time we call `next` on the generator, it `yield`s another batch. Neural networks are trained incrementally - a batch at a time - which means that a generator is a useful function for returning examples on which to train. This alleviates the need to store all of the training data in memory which might be an issue if we were working with a larger dataset such as images. 

In [95]:
import numpy as np
import random
random.seed(100)

def generate_batch(pairs, n_positive = 50, negative_ratio = 1.0, classification = False):
    """Generate batches of samples for training"""
    batch_size = n_positive * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    
    # Adjust label based on task
    if classification:
        neg_label = 0
    else:
        neg_label = -1
    
    # This creates a generator
    while True:
        # randomly choose positive examples
        for idx, (book_id, link_id) in enumerate(random.sample(pairs, n_positive)):
            batch[idx, :] = (book_id, link_id, 1)

        # Increment idx by 1
        idx += 1
        
        # Add negative examples until reach batch size
        while idx < batch_size:
            
            # random selection
            random_book = random.randrange(len(books))
            random_link = random.randrange(len(links))
            
            # Check to make sure this is not a positive example
            if (random_book, random_link) not in pairs_set:
                
                # Add to batch and increment index
                batch[idx, :] = (random_book, random_link, neg_label)
                idx += 1
                
        # Make sure to shuffle order
        np.random.shuffle(batch)
        yield {'book': batch[:, 0], 'link': batch[:, 1]}, batch[:, 2]

To get a new batch, call `yield` on the generator. 

In [96]:
next(generate_batch(pairs, n_positive = 2, negative_ratio = 2))

({'book': array([ 6917., 25757., 28410., 22239., 29814.,  7206.]),
  'link': array([21344., 22920., 33217.,  6046., 11452., 34924.])},
 array([ 1., -1., -1.,  1., -1., -1.]))

In [97]:
x, y = next(generate_batch(pairs, n_positive = 2, negative_ratio = 2))

for lab, b_idx, l_idx in zip(y, x['book'], x['link']):
    print(f'Book: {index_book[b_idx]:25} Link: {index_link[l_idx]:40} Label: {lab}') 

Book: Des Imagistes             Link: journal of asian studies                 Label: -1.0
Book: Soul Music (novel)        Link: bermuda triangle                         Label: -1.0
Book: Black Skin, White Masks   Link: nationalism                              Label: 1.0
Book: The Soul of the Robot     Link: allanon                                  Label: -1.0
Book: The Counterfeit Man       Link: julie andrews                            Label: -1.0
Book: Lord of the World         Link: prime minister of the united kingdom     Label: 1.0


# Neural Network Embedding Model

With our training samples generator, we're almost there. The next step is the most technically complicated but thankfully made much easier with Keras. We are going to construct the neural network that learns the entity embeddings. The input to this network is the [book, link] as integers, and the output will be a prediction of whether or not the link was present in the book article. The heart of the network is the embedding layers, one for the book and one for the link each of which maps the input entity to a 50 dimensional vector. 

After converting to the embedding, we need a way to combine the embeddings into a single number. For this we can use the dot product which does element-wise multiplication of numbers in the vectors and then sums the result to a single number. This number is then the ouput of the model for the case of regression. In regression, our labels are either -1 or 1, and so the model loss function will be mean squared error in order to minimize the distance between the prediction and the output. 

For classification, we add an extra fully connected `Dense` layer with a `sigmoid` activation to squash the outputs between 0 and 1 because the labels are either 0 or 1. The loss function for classification is `binary_crossentropy` which measures the [error of the neural network predictions in a binary classification problem](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html), and is a measure of the similarity between two distributions. 

The optimizer - the algorithm used to update the parameters of the neural network through backpropagation - is Adam in both cases. We use the default parameters for this optimizer. 

In [26]:
from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model

C:\Users\willk\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [126]:
def book_embedding_model(embedding_size = 50, classification = False):
    """Model to embed books and wikilinks using the functional API.
       Trained to discern if a link is present in a article"""
    
    # Both inputs are 1-dimensional
    book = Input(name = 'book', shape = [1])
    link = Input(name = 'link', shape = [1])
    
    # Embedding the book
    book_embedding = Embedding(name = 'book_embedding',
                               input_dim = len(book_index),
                               output_dim = embedding_size)(book)
    
    # Embedding the link
    link_embedding = Embedding(name = 'link_embedding',
                               input_dim = len(link_index),
                               output_dim = embedding_size)(link)
    
    # Reshape to be a single embedding size vector
    book_embedding = Reshape(target_shape = [embedding_size])(book_embedding)
    link_embedding = Reshape(target_shape = [embedding_size])(link_embedding)
    
    # Merge the layers with a dot product
    merged = Dot(name = 'dot_product', normalize = True, axes = 1)([book_embedding, link_embedding])
    
    # If classifcation, add extra layer and loss function is binary cross entropy
    if classification:
        merged = Dense(1, activation = 'sigmoid')(merged)
        model = Model(inputs = [book, link], outputs = merged)
        model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    # Otherwise loss function is mean squared error
    else:
        model = Model(inputs = [book, link], outputs = merged)
        model.compile(optimizer = 'Adam', loss = 'mse')
    
    return model

# Instantiate model and show parameters
model = book_embedding_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
book (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
link (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
book_embedding (Embedding)      (None, 1, 50)        1851000     book[0][0]                       
__________________________________________________________________________________________________
link_embedding (Embedding)      (None, 1, 50)        1927500     link[0][0]                       
__________________________________________________________________________________________________
reshape_3 

We can see that during training, there are nearly 3.8 million weights that need to be learned by the neural network. Each of these represents one number in an embedding for one entity. 

# Train Model

We have the training data - in a generator - and a model. The next step is to train the model to learn the entity embeddings. During this process, the model will update the embeddings in order to accomplish the task or predicting whether a certain link is actually on a book page or not. The resulting embeddings can then be used as a representation of books and links. 

There are a few parameters to adjust for training. The batch size should generally be as large as possible given the memory constraints of your machine. The negative ratio can be adjusted based on results. I tried 2 and it seemed to work well. The number of steps per epoch is chosen such that the model sees a number of examples equal to the number of pairs on each epoch. This is repeated for 20 epochs (which might be more than necessary).



In [98]:
n_positive = 512

gen = generate_batch(pairs, n_positive, negative_ratio = 5)

model.fit_generator(gen, epochs = 20, 
                    steps_per_epoch = len(pairs) // n_positive,
                    verbose = 2)

Epoch 1/20
 - 31s - loss: 0.3162
Epoch 2/20
 - 31s - loss: 0.3149
Epoch 3/20
 - 32s - loss: 0.3179
Epoch 4/20
 - 32s - loss: 0.3193
Epoch 5/20
 - 32s - loss: 0.3171
Epoch 6/20
 - 31s - loss: 0.3161
Epoch 7/20
 - 32s - loss: 0.3158
Epoch 8/20
 - 31s - loss: 0.3141
Epoch 9/20
 - 31s - loss: 0.3132
Epoch 10/20
 - 32s - loss: 0.3128
Epoch 11/20
 - 31s - loss: 0.3129
Epoch 12/20
 - 32s - loss: 0.3144
Epoch 13/20
 - 32s - loss: 0.3124
Epoch 14/20
 - 32s - loss: 0.3136
Epoch 15/20
 - 34s - loss: 0.3149
Epoch 16/20
 - 36s - loss: 0.3135
Epoch 17/20
 - 35s - loss: 0.3128
Epoch 18/20
 - 37s - loss: 0.3140
Epoch 19/20
 - 35s - loss: 0.3120
Epoch 20/20
 - 38s - loss: 0.3122


The entire trained model can be saved and later loaded in. It's also possible to save certain layers.

In [99]:
model.save('first_attempt.h5')

## Extract Embeddings

The trained model has learned - hopefully - representations of books and wikilinks that place similar entities next to one another. To find out if this is the case, we extract the embeddings and use them to find similar books and links.

In [100]:
# Extract embeddings
book_layer = model.get_layer('book_embedding')
book_weights = book_layer.get_weights()[0]
book_weights.shape

(37020, 50)

We need to normalize the embeddings so that the dot product between two embeddings becomes the cosine similarity.

In [101]:
book_weights = book_weights / np.linalg.norm(book_weights, axis = 1).reshape((-1, 1))
np.sum(np.square(book_weights[0]))

1.0

Normalize just means divide each vector by the square root of the sum of squared components.

### Function to Find Most Similar Entities

The function below takes in either a book or a link, a set of embeddings, and returns the `n` most similar items to the query. It does this by computing the dot product between the query and embeddings. Because we normalized the embeddings, the dot product represents the cosine similarity between two vectors. This is a measure of similarity that does not depend on the magnitude of the vector as does the Euclidean distance. The Euclidean distance would be another valid metric of similary to use to compare the embeddings.

In [111]:
def find_similar(name, weights, index_name = 'book', n = 10, least = False, return_dist = False):
    """Find n most similar items (or least) to name based on embeddings"""
    
    # Select index and reverse index
    if index_name == 'book':
        index = book_index
        rindex = index_book
    elif index_name == 'link':
        index = link_index
        rindex = index_link
    
    # Check to make sure `name` is in index
    try:
        # Calculate dot product between book and all others
        dists = np.dot(weights, weights[index[name]])
    except KeyError:
        print(f'{name} Not Found.')
        return
    
    # If specified, find the least similar
    if least:
        # Sort the indices from smallest to largest and take the first n
        closest = np.argsort(dists)[:n]
        
    # Otherwise find the most similar
    else:
        # Sort the indices from smallest to largest and take the last n
        closest = np.argsort(dists)[-n:]
    
    # Need distances later on
    if return_dist:
        return dists, closest
    
    # Print formatting
    max_width = max([len(rindex[c]) for c in closest])
    
    # Print the most similar and distances
    for c in reversed(closest):
        print(f'{index_name.capitalize()}: {rindex[c]:{max_width + 2}} Similarity: {dists[c]:.{2}}')

We can know that this function works if the most similar book is the book itself. Because we multiply the item vector times all the other embeddings, the most similar should be the item itself with a similarity of 1.0.

In [112]:
find_similar('War and Peace', book_weights)

Book: War and Peace               Similarity: 1.0
Book: Anna Karenina               Similarity: 0.95
Book: The Master and Margarita    Similarity: 0.91
Book: Candide                     Similarity: 0.89
Book: Poor Folk                   Similarity: 0.88
Book: Demons (Dostoevsky novel)   Similarity: 0.88
Book: Buddenbrooks                Similarity: 0.87
Book: Crime and Punishment        Similarity: 0.86
Book: Dead Souls                  Similarity: 0.85
Book: The Good Soldier Švejk      Similarity: 0.85


It works! The most similar books make sense at least for War and Peace.

In [113]:
find_similar('War and Peace', book_weights, least = True, n = 5)

Book: Crystal Mask                 Similarity: -0.35
Book: List of Transformers books   Similarity: -0.36
Book: Dungeon Master's Guide       Similarity: -0.38
Book: Kushiel's Justice            Similarity: -0.39
Book: Player's Handbook            Similarity: -0.4


The least similar books are quite a grab bag!

In [114]:
find_similar('The Autobiography of Benjamin Franklin', book_weights, n = 5)

Book: The Autobiography of Benjamin Franklin           Similarity: 1.0
Book: Pudd'nhead Wilson                                Similarity: 0.94
Book: Cloudsplitter                                    Similarity: 0.93
Book: Poems on Various Subjects, Religious and Moral   Similarity: 0.93
Book: Letters from an American Farmer                  Similarity: 0.93


It doesn't work perfectly for every book as we can see.

In [115]:
find_similar('A More Perfect Constitution', book_weights, n = 5)

Book: A More Perfect Constitution                            Similarity: 1.0
Book: Never Trust a Liberal Over 3—Especially a Republican   Similarity: 0.99
Book: Government Bullies                                     Similarity: 0.99
Book: National Security and Double Government                Similarity: 0.99
Book: Culture of Corruption                                  Similarity: 0.99


In [116]:
find_similar('The Fellowship of the Ring', book_weights, n = 5)

Book: The Fellowship of the Ring   Similarity: 1.0
Book: The Return of the King       Similarity: 0.98
Book: Beren and Lúthien            Similarity: 0.96
Book: The Children of Húrin        Similarity: 0.95
Book: The Silmarillion             Similarity: 0.95


In [117]:
find_similar('The Two Towers', book_weights, n = 5)

Book: The Two Towers                Similarity: 1.0
Book: Beren and Lúthien             Similarity: 0.96
Book: Morgoth's Ring                Similarity: 0.96
Book: The History of Middle-earth   Similarity: 0.96
Book: The Book of Lost Tales        Similarity: 0.96


Not bad by the looks of the results! I encourage you to play around with the model and explore the resulting embeddings. 

In [118]:
find_similar('Bully for Brontosaurus', book_weights, n = 5)

Book: Bully for Brontosaurus          Similarity: 1.0
Book: I Have Landed                   Similarity: 0.98
Book: An Urchin in the Storm          Similarity: 0.97
Book: Eight Little Piggies            Similarity: 0.97
Book: Biology for Christian Schools   Similarity: 0.97


## Link Embeddings

We also have the embeddings of wikipedia links. We can take a similar approach to extract these and find the most similar to a query link. 

Let's write a quick function to extract weights from a model given the name.

In [119]:
def extract_weights(name, model):
    """Extract weights from a neural network model"""
    
    # Extract weights
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    
    # Normalize
    weights = weights / np.linalg.norm(weights, axis = 1).reshape((-1, 1))
    return weights

In [120]:
link_weights = extract_weights('link_embedding', model)

We can use the same find similar function to find the most similar links to a given link.

In [121]:
find_similar('science fiction', link_weights, index_name = 'link')

Link: science fiction                            Similarity: 1.0
Link: category:american science fiction novels   Similarity: 0.94
Link: fantasy                                    Similarity: 0.88
Link: tor books                                  Similarity: 0.85
Link: united states                              Similarity: 0.85
Link: english language                           Similarity: 0.83
Link: category:doubleday (publisher) books       Similarity: 0.83
Link: doubleday (publisher)                      Similarity: 0.83
Link: novel                                      Similarity: 0.83
Link: bantam books                               Similarity: 0.8


In [122]:
find_similar('biography', link_weights, index_name = 'link')

Link: biography                             Similarity: 1.0
Link: autobiography                         Similarity: 0.9
Link: category:american non-fiction books   Similarity: 0.86
Link: non-fiction                           Similarity: 0.85
Link: category:english-language books       Similarity: 0.83
Link: category:2005 books                   Similarity: 0.83
Link: memoir                                Similarity: 0.83
Link: category:2010 non-fiction books       Similarity: 0.81
Link: category:2008 books                   Similarity: 0.8
Link: category:2006 books                   Similarity: 0.8


In [123]:
find_similar('biography', link_weights, index_name = 'link', least = True, n = 5)

Link: eos books                             Similarity: -0.76
Link: midnighters trilogy                   Similarity: -0.76
Link: sword                                 Similarity: -0.76
Link: category:novels by scott westerfeld   Similarity: -0.76
Link: the uglies series                     Similarity: -0.79


Now you not only know what books to look for, you know the books to avoid for a given category! 

In [124]:
find_similar('new york city', link_weights, index_name = 'link', n = 5)

Link: new york city                                 Similarity: 1.0
Link: alfred a. knopf                               Similarity: 0.93
Link: the new york times                            Similarity: 0.92
Link: simon  &  schuster                            Similarity: 0.91
Link: category:american novels adapted into films   Similarity: 0.9


## Classification Model

I was curious if training for the mean squared error as a regression problem was the ideal approach, so I also decided to experiment with a classifcation model. For this model, the negative examples receive a label of 0 and the loss function is binary cross entropy. The procedure for the neural network to learn the embeddings is exactly the same, only it will be optimizing for a slightly different measure.

In [128]:
model_acc = book_embedding_model(50, classification = True)
gen = generate_batch(pairs, n_positive, negative_ratio=2, classification = True)

The previous model might not have needed to train for so long. We can reduce the number of epochs and see if the result is affected.

In [ ]:
# Train the model to learn embeddings
h = model_acc.fit_generator(gen, epochs = 10, steps_per_epoch= len(pairs) // n_positive,
                            verbose = 0)

We'll go through the same process, extracting the weights and finding similar books based on the embedding space representation.

In [ ]:
model.save('first_attempt_class.h5')

In [ ]:
book_weights_acc = extract_weights('book_embedding', model_acc)
book_weights_acc.shape

In [ ]:
find_similar('War and Peace', book_weights_acc)

In [ ]:
find_similar('The Fellowship of the Ring', book_weights_acc)

In [ ]:
link_weights_acc = extract_weights('link_embedding', model_acc)

In [ ]:
find_similar('new york times', link_weights_acc, index_name = 'link', n = 5)

In [ ]:
find_similar('category:1964 fantasy novels', link_weights_acc, index_name = 'link', n = 5)

# Visualizations

One of the coolest part about embeddings is that we can visualize them. First we have to take them from 50 dimensions down to either 3 or 2. We can do this using `pca`, `tsne`, or `umap`. We'll try both tsne and umap for comparison.

In [ ]:
from sklearn.manifold import TSNE
from umap import UMAP

In [ ]:
def reduce_dim(weights, components = 3, method = 'tsne'):
    if method == 'tsne':
        return TSNE(components).fit_transform(weights)
    elif method == 'umap':
        return UMAP(n_components=components, metric = 'cosine', 
                    init = 'random', n_neighbors = 2).fit_transform(weights)

In [ ]:
book_r = reduce_dim(book_weights_acc, components = 2, method = 'tsne')
book_r.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

plt.plot(book_r[:, 0], book_r[:, 1], 'r.')

In [ ]:
book_ru = reduce_dim(book_weights_acc, components = 2, method = 'umap')
plt.plot(book_ru[:, 0], book_ru[:, 1], 'g.');

### Book Embeddings by Category

This is a little hard to interpret. Let's take the 10 most popular categories, and color the image by the category.

In [ ]:
categories = [[link for link in book[2] if 'Category:' in link] for book in books]
c_counts = count_items(list(chain(*categories)))
list(c_counts.items())[:5]

We'll stick to the 10 most popular categories. 

In [ ]:
c_to_include = list(c_counts.keys())[:10]
c_to_include = [c.lower() for c in c_to_include]
c_to_include[:5]

A lot of these categories have overlap and books can have multiple categories. Therefore we'll just take the first category for the book that is in the categories to include.

In [ ]:
idx_include = []
cs = []

# Iterate through each book's categories
for i, book_c in enumerate(categories):
    # Iterate through books categories in the book
    for c in book_c:
        # If link is in c to include, record the index of book and category
        if c.lower() in c_to_include:
            idx_include.append(i)
            cs.append(c.lower())
            break

Now we need to map the categories to integers. 

In [ ]:
ints, cats = pd.factorize(cs)
cats

Finally, we can plot the embedding colored by the category. 

In [ ]:
plt.figure(figsize = (10, 8))

# Plot embedding
plt.scatter(book_ru[idx_include, 0], book_ru[idx_include, 1], 
            c = ints, cmap = plt.cm.tab10)

# Add colorbar and appropriate labels
cbar = plt.colorbar()
cbar.set_ticks(list(range(10)))
cbar.set_ticklabels(cats)


plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.title('UMAP Visualization of Book Embeddings');

There doesn't appear to be much separation between the categories. There are a lot of parameters to play around with in UMAP, and changing some of them might result in better clusters.

As a last step, let's see the embedding labeled with the 10 books most often mentioned by other books.

In [ ]:
wikilink_books = list(chain(*[[l for l in book[2] if l in book_index.keys()] for book in books]))
wikilink_book_counts = count_items(wikilink_books)
list(wikilink_book_counts.items())[:10]

In [ ]:
InteractiveShell.ast_node_interactivity = 'last'

plt.figure(figsize = (10, 8))

# Plot embedding
plt.scatter(book_ru[idx_include, 0], book_ru[idx_include, 1], 
            c = ints, cmap = plt.cm.tab10, alpha = 0.4)

# Add colorbar and appropriate labels
cbar = plt.colorbar()
cbar.set_ticks(list(range(10)))
cbar.set_ticklabels(cats)


plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.title('UMAP Visualization of Book Embeddings');

for book in list(wikilink_book_counts.keys())[:10]:
    
    x, y = book_ru[book_index[book], 0], book_ru[book_index[book], 1];
    _ = plt.scatter(x, y, s = 200, marker = '*', edgecolor = 'k')
    _ = plt.text(x, y, book, fontsize = 12);


## Plot Book Nearest Neighbors

To get a better sense of which books are located where, we can plot a book along with its nearest neighbors. These will be the nearest neighbors in the original embedding space, so they are not necessarily the closest in the reduced dimension representation.

In [ ]:
dist, closest = find_similar('War and Peace', book_weights_acc, 'book', n = 10, return_dist=True)

In [ ]:
def plot_closest(item, weights, index_name, n, plot_data):
    """Plot n most closest items to item"""
    
    # Find the closest items
    dist, closest = find_similar(item, weights, index_name, n, return_dist=True)
    
    # Choose mapping for look up
    if index_name == 'book':
        index = book_index
        rindex = index_book
    elif index_name == 'link':
        index = link_index
        rindex = index_link 
    
    plt.figure(figsize = (10, 8))
    plt.rcParams['font.size'] = 14
    
    # Limit distances
    dist = dist[closest]
    
    # Plot all of the data
    plt.scatter(plot_data[:, 0], plot_data[:, 1], alpha = 0.1, color = 'goldenrod')
    
    # Plot the item
    plt.scatter(plot_data[closest[-1], 0], plot_data[closest[-1], 1], s = 600, edgecolor = 'k', color = 'b')
    
    # Plot the closest items
    plt.scatter(plot_data[closest, 0], plot_data[closest, 1], 
                c = dist, cmap = plt.cm.copper, s = 200, alpha = 1, marker = '*')
    
    # Colorbar management
    cbar = plt.colorbar()
    cbar.set_label(f'{index_name.capitalize()} and Cosine Distance')
    cbar.set_ticks(np.linspace(dist.min(), dist.max(), 10))
    cbar.set_ticklabels([f'{rindex[idx]}: {distance:.2}' for idx, distance in zip(closest, dist)])
    
    # Labeling
    plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.title(f'Most Similar {index_name.capitalize()}s to {item}')

In [ ]:
plot_closest('War and Peace', book_weights_acc, 'book', 10, book_ru)

In [ ]:
plot_closest('A Brief History of Time', book_weights_acc, 'book', 10, book_ru)

## Embedding Links

Let's look at reducing the dimension of the embedding for the links. We'll start with only the categories.

In [ ]:
idx_to_include = [idx for link, idx in link_index.items() if 'category:' in link]
link_ru = reduce_dim(link_weights_acc[idx_to_include], components = 2, method = 'umap')
link_ru.shape

We'll get the 10 most popular categories so we can plot them highlighted.

In [ ]:
idx = []
for link in cats:
    idx.append(int(np.where(np.array(idx_to_include) == link_index[link.lower()])[0]))

In [ ]:
plt.figure(figsize = (12, 12))
plt.scatter(link_ru[:, 0], link_ru[:, 1], alpha = 0.6)

for i in idx:
    x, y = link_ru[i, 0], link_ru[i, 1]
    s = index_link[idx_to_include[i]].split(':')[-1]
    _ = plt.text(x, y, s, fontsize = 18);
    
plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.title('UMAP Visualization of Wikilinks Embedding');

In [ ]:
plot_closest('category:american science fiction novels', link_weights_acc, 'link', 10, link_ru)

In [ ]:
plot_closest('category:novels set in new york city', link_weights_acc, 'link', 10, link_ru)

Now we can go ahead and reduce the dimension of all the links. Then we can do the same as before, plotting the closest links.

In [ ]:
link_ru = reduce_dim(link_weights_acc, components = 2, method = 'umap')
link_ru.shape

In [ ]:
plot_closest('new york times', link_weights_acc, 'link', 10, link_ru)

In [ ]:
plot_closest('james joyce', link_weights_acc, 'link', 10, link_ru)

In [ ]:
plot_closest('margaret atwood', link_weights_acc, 'link', 10, link_ru)

In [ ]:
plot_closest('george eliot', link_weights_acc, 'link', 10, link_ru)

# Save Embeddings for Visualization

We can save these embeddings for visualization in [projector.tensorflow.org](https://projector.tensorflow.org/)

In [ ]:
%%capture
with open('link_names.tsv', 'w' , encoding = 'utf-8') as fout:
    for l in link_index.keys():
        fout.write(str(l))
        fout.write('\n')

In [ ]:
%%capture
with open('book_names.tsv', 'w' , encoding = 'utf-8') as fout:
    for l in book_index.keys():
        fout.write(str(l))
        fout.write('\n')

The book embeddings and link embeddings can be saved as tab separated files (the format specified by the online application). 

In [ ]:
np.savetxt('book_embedding.tsv', book_weights_acc, delimiter='\t')
np.savetxt('link_embedding.tsv', link_weights_acc, delimiter='\t')

In [ ]:
book_embedding = np.loadtxt('book_embedding.tsv', delimiter = '\t')
book_embedding.shape

# Conclusions

In this notebook, we built an effective book recommendation system using the outgoing Wikipedia links from all books articles on Wikipedia. We did this by creating embeddings of the books and the links with the assumption that books linking to similar pages are similar to one another. We saw how to inspect the embeddings in order to find the closest books or links and also how to visualize the embeddings which sometimes can show us interestings clusterings. The next step is to visualize these embeddings in an interactive tool, or use additional information to refine the book recommendation system.